<a href="https://colab.research.google.com/github/half-glass/deep_week3/blob/master/CNN_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
required_training = True
%matplotlib inline
import matplotlib.pyplot as plt  # one of the best graphics library for python
import os
import time

from typing import Iterable
from dataclasses import dataclass

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision import datasets, transforms
import torchvision

def get_mean_std_train_data(data_root):
    
    train_transform = transforms.Compose([transforms.ToTensor()])
    train_set = datasets.CIFAR10(root=data_root, train=True, download=False, transform=train_transform)
    
    # return mean (numpy.ndarray) and std (numpy.ndarray)
    mean = np.array([0.5, 0.5, 0.5])
    std = np.array([0.5, 0.5, 0.5])
    
    ###
    d = torch.tensor(train_set.data, dtype=float)
    #print(d.shape)
    #print(d.dtype)
    mean = torch.mean(d / 255., [0, 1, 2])
    std = torch.std(d / 255., [0, 1, 2])
    #mean = np.array([0., 0., 0.])
    #std = np.array([1., 1., 1.])
    #mean = np.array([128., 128., 128.])
    #std = np.array([25., 25., 25.])
    ###
    
    return mean, std

def get_data(batch_size, data_root, num_workers=1):
    
    
    try:
        mean, std = get_mean_std_train_data(data_root)
        assert len(mean) == len(std) == 3
    except:
        mean = np.array([0.5, 0.5, 0.5])
        std = np.array([0.5, 0.5, 0.5])
        
    
    train_test_transforms = transforms.Compose([                     
        # this re-scale image tensor values between 0-1. image_tensor /= 255
        transforms.ToTensor(),
        # subtract mean and divide by variance.
        transforms.Normalize(mean, std),
        transforms.RandomChoice([transforms.RandomHorizontalFlip(),
                                 transforms.RandomRotation(10),
                                 ])
    ])
    
    # train dataloader
    train_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10(root=data_root, train=True, download=True, transform=train_test_transforms),
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers
    )
    
    # test dataloader
    test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10(root=data_root, train=False, download=False, transform=train_test_transforms),
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers
    )
    return train_loader, test_loader

@dataclass
class SystemConfiguration:
    '''
    Describes the common system setting needed for reproducible training
    '''
    seed: int = 42  # seed number to set the state of all random number generators
    cudnn_benchmark_enabled: bool = True  # enable CuDNN benchmark for the sake of performance
    cudnn_deterministic: bool = True  # make cudnn deterministic (reproducible training)

def setup_system(system_config: SystemConfiguration) -> None:
    torch.manual_seed(system_config.seed)
    if torch.cuda.is_available():
        torch.backends.cudnn_benchmark_enabled = system_config.cudnn_benchmark_enabled
        torch.backends.cudnn.deterministic = system_config.cudnn_deterministic

@dataclass
class TrainingConfiguration:
    '''
    Describes configuration of the training process
    '''
    batch_size: int = 16  # amount of data to pass through the network at each forward-backward iteration
    epochs_count: int = 2  # number of times the whole dataset will be passed through the network
    learning_rate: float = 0.1  # determines the speed of network's weights update
        
    log_interval: int = 100  # how many batches to wait between logging training status
    test_interval: int = 1  # how many epochs to wait before another test. Set to 1 to get val loss at each epoch
    data_root: str = "../resource/lib/publicdata/images"  # folder to save data
    num_workers: int = 2  # number of concurrent processes using to prepare data
    device: str = 'cuda'  # device to use for training.
    # update changed parameters in blow coding block.
    # Please do not change "data_root" 
    
    ###
    batch_size: int = 64
    epochs_count: int = 30
    learning_rate: float = 0.0006
    ###

def train(
    train_config: TrainingConfiguration, model: nn.Module, optimizer: torch.optim.Optimizer,
    train_loader: torch.utils.data.DataLoader, epoch_idx: int
) -> None:
    
    # change model in training mood
    model.train()
    
    # to get batch loss
    batch_loss = np.array([])
    
    # to get batch accuracy
    batch_acc = np.array([])
    
    # total correct
    correctSum = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        
        # clone target
        indx_target = target.clone()
        # send data to device (its is medatory if GPU has to be used)
        data = data.to(train_config.device)
        # send target to device
        target = target.to(train_config.device)

        # reset parameters gradient to zero
        optimizer.zero_grad()
        
        # forward pass to the model
        output = model(data)
        
        # cross entropy loss
        loss = F.cross_entropy(output, target)
        
        # find gradients w.r.t training parameters
        loss.backward()
        # Update parameters using gardients
        optimizer.step()
        
        batch_loss = np.append(batch_loss, [loss.item()])
        
        # Score to probability using softmax
        prob = F.softmax(output, dim=1)
            
        # get the index of the max probability
        pred = prob.data.max(dim=1)[1]  
                        
        # correct prediction
        correct = pred.cpu().eq(indx_target).sum()
        correctSum += correct
            
        # accuracy
        acc = float(correct) / float(len(data))
        
        batch_acc = np.append(batch_acc, [acc])

        #if batch_idx % train_config.log_interval == 0 and batch_idx > 0:              
        #    print(
        #        'Train Epoch: {} [{}/{}] Loss: {:.6f} Acc: {:.4f}'.format(
        #            epoch_idx, batch_idx * len(data), len(train_loader.dataset), loss.item(), acc
        #        )
        #    )
    
    epoch_loss = batch_loss.mean()
    epoch_acc = batch_acc.mean()

    print(
        'Trainset: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
            epoch_loss, correctSum, len(train_loader.dataset), epoch_acc*100
        )
    )

    return epoch_loss, epoch_acc

def validate(
    train_config: TrainingConfiguration,
    model: nn.Module,
    test_loader: torch.utils.data.DataLoader,
) -> float:
    model.eval()
    test_loss = 0
    count_corect_predictions = 0
    for data, target in test_loader:
        indx_target = target.clone()
        data = data.to(train_config.device)
        
        target = target.to(train_config.device)
        
        output = model(data)
        # add loss for each mini batch
        test_loss += F.cross_entropy(output, target).item()
        
        # Score to probability using softmax
        prob = F.softmax(output, dim=1)
        
        # get the index of the max probability
        pred = prob.data.max(dim=1)[1] 
        
        # add correct prediction count
        count_corect_predictions += pred.cpu().eq(indx_target).sum()

    # average over number of mini-batches
    test_loss = test_loss / len(test_loader)  
    
    # average over number of dataset
    accuracy = 100. * count_corect_predictions / len(test_loader.dataset)
    
    print(
        'Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
            test_loss, count_corect_predictions, len(test_loader.dataset), accuracy
        )
    )
    return test_loss, accuracy/100.0

def save_model(model, device, model_dir='models', model_file_name='cifar10_cnn_model.pt'):
    

    if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    model_path = os.path.join(model_dir, model_file_name)

    # make sure you transfer the model to cpu.
    if device == 'cuda':
        model.to('cpu')

    # save the state_dict
    torch.save(model.state_dict(), model_path)
    
    if device == 'cuda':
        model.to('cuda')
    
    return

def main(system_configuration=SystemConfiguration(), training_configuration=TrainingConfiguration()):
    
    # system configuration
    setup_system(system_configuration)

    # batch size
    batch_size_to_set = training_configuration.batch_size
    # num_workers
    num_workers_to_set = training_configuration.num_workers
    # epochs
    epoch_num_to_set = training_configuration.epochs_count

    # if GPU is available use training config, 
    # else lowers batch_size, num_workers and epochs count
    if torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"
        num_workers_to_set = 2

    # data loader
    train_loader, test_loader = get_data(
        batch_size=training_configuration.batch_size,
        data_root=training_configuration.data_root,
        num_workers=num_workers_to_set
    )
    
    # Update training configuration
    training_configuration = TrainingConfiguration(
        device=device,
        num_workers=num_workers_to_set
    )

    # initiate model
    model = MyModel()
        
    # send model to device (GPU/CPU)
    model.to(training_configuration.device)

    # optimizer
    #optimizer = optim.SGD(
    #    model.parameters(),
    #    lr=training_configuration.learning_rate,
    #    momentum = 0.9,
    #)
    optimizer = optim.Adam(
        model.parameters(),
        lr=training_configuration.learning_rate,
    )

    best_loss = torch.tensor(np.inf)
    best_accuracy = torch.tensor(0)
    
    # epoch train/test loss
    epoch_train_loss = np.array([])
    epoch_test_loss = np.array([])
    
    # epch train/test accuracy
    epoch_train_acc = np.array([])
    epoch_test_acc = np.array([])
    
    # trainig time measurement
    t_begin = time.time()
    for epoch in range(training_configuration.epochs_count):
        
        train_loss, train_acc = train(training_configuration, model, optimizer, train_loader, epoch)
        
        epoch_train_loss = np.append(epoch_train_loss, [train_loss])
        
        epoch_train_acc = np.append(epoch_train_acc, [train_acc])

        elapsed_time = time.time() - t_begin
        speed_epoch = elapsed_time / (epoch + 1)
        speed_batch = speed_epoch / len(train_loader)
        eta = speed_epoch * training_configuration.epochs_count - elapsed_time
        
        #print(
        #    "Elapsed {:.2f}s, {:.2f} s/epoch, {:.2f} s/batch, ets {:.2f}s".format(
        #        elapsed_time, speed_epoch, speed_batch, eta
        #    )
        #)

        if epoch % training_configuration.test_interval == 0:
            current_loss, current_accuracy = validate(training_configuration, model, test_loader)
            
            epoch_test_loss = np.append(epoch_test_loss, [current_loss])
        
            epoch_test_acc = np.append(epoch_test_acc, [current_accuracy])
            
            if current_loss < best_loss:
                best_loss = current_loss
            
            if current_accuracy > best_accuracy:
                best_accuracy = current_accuracy
                print('Accuracy improved, saving the model.\n')
                save_model(model, device)
            
                
    print("Total time: {:.2f}, Best Loss: {:.3f}, Best Accuracy: {:.3f}".format(time.time() - t_begin, best_loss, 
                                                                                best_accuracy))
    
    return model, epoch_train_loss, epoch_train_acc, epoch_test_loss, epoch_test_acc

class MyModel(nn.Module):
    def __init__(self):
        super().__init__()

        ###
        k = 3
        p = 1
        z1 = 256 #32 #64
        z2 = 512 #32 #128
        self._body = nn.Sequential(
            # input size = (32, 32, 3)
            
            # 32 --> imSize /= 2 --> 16
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=k, padding=p, padding_mode='replicate'),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=k, padding=p, padding_mode='replicate'),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),

            # 16 --> imSize /= 2 --> 8
            nn.Conv2d(in_channels=32, out_channels=z1, kernel_size=k, padding=p, padding_mode='replicate'),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=z1, out_channels=z1, kernel_size=k, padding=p, padding_mode='replicate'),
            nn.BatchNorm2d(z1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.5),

            # 8 --> imSize /= 2 --> 4
            nn.Conv2d(in_channels=z1, out_channels=z1, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=z1, out_channels=z2, kernel_size=k, padding=p, padding_mode='replicate'),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=z2, out_channels=z2, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=z2, out_channels=z2, kernel_size=k, padding=p, padding_mode='replicate'),
            nn.BatchNorm2d(z2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.5),

            #nn.BatchNorm2d(32),
        )
        
        self._head = nn.Sequential(
            nn.Linear(in_features=z2*4*4, out_features=120),
            nn.ReLU(inplace=True),
            
            #nn.Linear(in_features=120, out_features=84),
            #nn.ReLU(inplace=True),
            
            nn.Linear(in_features=120, out_features=10)
        )
        ###
        
    def forward(self, x):
        ###
        x = self._body(x)
        x = x.view(x.size()[0], -1)
        x = self._head(x)
        ###
        
        return x



my_model = MyModel()
#print(my_model)

if required_training:
    model, epoch_train_loss, epoch_train_acc, epoch_test_loss, epoch_test_acc = main()

Files already downloaded and verified
Trainset: Average loss: 1.4077, Accuracy: 24077/50000 (48%)
Test set: Average loss: 1.1357, Accuracy: 5857/10000 (59%)
Accuracy improved, saving the model.

Trainset: Average loss: 1.0117, Accuracy: 31882/50000 (64%)
Test set: Average loss: 1.2430, Accuracy: 5861/10000 (59%)
Accuracy improved, saving the model.

Trainset: Average loss: 0.8546, Accuracy: 34782/50000 (70%)
Test set: Average loss: 0.8223, Accuracy: 7148/10000 (71%)
Accuracy improved, saving the model.

Trainset: Average loss: 0.7648, Accuracy: 36457/50000 (73%)
Test set: Average loss: 0.8416, Accuracy: 7142/10000 (71%)
Trainset: Average loss: 0.6933, Accuracy: 37759/50000 (76%)
Test set: Average loss: 0.6833, Accuracy: 7646/10000 (76%)
Accuracy improved, saving the model.

Trainset: Average loss: 0.6476, Accuracy: 38722/50000 (77%)
Test set: Average loss: 0.6193, Accuracy: 7844/10000 (78%)
Accuracy improved, saving the model.

Trainset: Average loss: 0.6044, Accuracy: 39510/50000 (79%